In [ ]:
using Plots
using FFTW

L = 100
N = 1000
M = 1000
dx = L/N
k0 = 2π/L
dt = 1e-3
x = LinRange(0, L - dx, N)
k = 2*π / L .* collect(1:(N÷2)+1)
k² = k.*k

F = plan_rfft(x)
B = plan_irfft(k*(1 + 0*im), N)

const V = cos.(k0 .* x)
# const V = zeros(N)
const FV = F*V
const v = 4 * B*(-im.*k.*FV)

function antialiasing!( F )
    F[(end-(N÷2)÷3)+1:end] .= 0.
end

function euler!(Fdφ, Fφ)
    Fdφ .= -k².*Fφ
    Fdφ .+= (-im.*k) .* (F*(v.*(B*Fφ)))
    Fφ .+= Fdφ.*dt
    antialiasing!(Fφ)
end


In [ ]:
dφ = zeros(N)
a = 2
# φ = 5 * exp.( - ((x .- (L/4)).^2) ./ (2*a^2) ) ./ (√(2π*a^2)) 
# φ = cos.(2*k0*x)
φ = zeros(N)
φ[100:250] .= 1.
plot(x, φ)
plot!(x, V)

In [ ]:
plot(x, v)

In [ ]:
Fφ = F*φ
Fdφ = F*dφ
plot(k, real(Fφ))
plot!(k, imag(Fφ))

In [ ]:
φ = B*Fφ
dφ = B*Fdφ
plot(x, φ)

In [ ]:
M = 50_000
c = "lightblue"
p = plot(x, φ, color=c)
for t in 1:M
    euler!(Fdφ, Fφ)
    if t%(M÷10) == 0
        φ .= B*Fφ
        plot!(x, φ, color=c)
        @assert !any(isnan.(Fφ))
    end
end
display(p)
φ = B*Fφ
dφ = B*Fdφ
plot(x, φ, ylims=(-1,1))
plot!(x, V)
 